Radek posted about this [here](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/discussion/309220), and linked to a GitHub repo with the code.

I just transferred that code here to Kaggle notebooks, that's all.

In [98]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [99]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

class Categorize(BaseEstimator, TransformerMixin):
    def __init__(self, min_examples=0):
        self.min_examples = min_examples
        self.categories = []
        
    def fit(self, X):
        for i in range(X.shape[1]):
            vc = X.iloc[:, i].value_counts()
            self.categories.append(vc[vc > self.min_examples].index.tolist())
        return self

    def transform(self, X):
        data = {X.columns[i]: pd.Categorical(X.iloc[:, i], categories=self.categories[i]).codes for i in range(X.shape[1])}
        return pd.DataFrame(data=data)


def calculate_apk(list_of_preds, list_of_gts):
    # for fast validation this can be changed to operate on dicts of {'cust_id_int': [art_id_int, ...]}
    # using 'data/val_week_purchases_by_cust.pkl'
    apks = []
    for preds, gt in zip(list_of_preds, list_of_gts):
        apks.append(apk(gt, preds, k=12))
    return np.mean(apks)

def eval_sub(sub_csv, skip_cust_with_no_purchases=True):
    sub=pd.read_csv(sub_csv)
    validation_set=pd.read_parquet('data/validation_ground_truth.parquet')

    apks = []

    no_purchases_pattern = []
    for pred, gt in zip(sub.prediction.str.split(), validation_set.prediction.str.split()):
        if skip_cust_with_no_purchases and (gt == no_purchases_pattern): continue
        apks.append(apk(gt, pred, k=12))
    return np.mean(apks)

In [100]:
import pandas as pd

transactions = pd.read_parquet('../input/transactions_train.parquet')
customers = pd.read_parquet('../input/customers.parquet')
articles = pd.read_parquet('../input/articles.parquet')

test_week = transactions.week.max() + 1
transactions = transactions[transactions.week > transactions.week.max() - 10]

# Generating candidates

### Last purchase candidates

In [101]:
c2weeks = transactions.groupby('customer_id')['week'].unique()
c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week
    
candidates_last_purchase = transactions.copy()
weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])
candidates_last_purchase.week=weeks

In [102]:
transactions

,t_dat,customer_id,article_id,price,sales_channel_id,week
29030503,2020-07-15,272412481300040,778064028,0.008458,1,95
29030504,2020-07-15,272412481300040,816592008,0.016932,1,95
29030505,2020-07-15,272412481300040,621381021,0.033881,1,95
29030506,2020-07-15,272412481300040,817477003,0.025407,1,95
29030507,2020-07-15,272412481300040,899088002,0.025407,1,95
...,...,...,...,...,...,...
31774722,2020-09-22,18439937050817258297,891591003,0.084729,2,104
31774723,2020-09-22,18439937050817258297,869706005,0.084729,2,104
31779097,2020-09-22,18440902715633436014,918894002,0.016932,1,104
31779098,2020-09-22,18440902715633436014,761269001,0.016932,1,104


In [103]:
candidates_last_purchase

,t_dat,customer_id,article_id,price,sales_channel_id,week
29030503,2020-07-15,272412481300040,778064028,0.008458,1,96
29030504,2020-07-15,272412481300040,816592008,0.016932,1,96
29030505,2020-07-15,272412481300040,621381021,0.033881,1,96
29030506,2020-07-15,272412481300040,817477003,0.025407,1,96
29030507,2020-07-15,272412481300040,899088002,0.025407,1,96
...,...,...,...,...,...,...
31774722,2020-09-22,18439937050817258297,891591003,0.084729,2,105
31774723,2020-09-22,18439937050817258297,869706005,0.084729,2,105
31779097,2020-09-22,18440902715633436014,918894002,0.016932,1,105
31779098,2020-09-22,18440902715633436014,761269001,0.016932,1,105


In [104]:
# generate personalized candidates for each customer based on three features: age, index_group_name and mean purchase price of the customer.

# first create mean price per customer
mean_price_per_c = transactions.groupby('customer_id')['price'].mean().rename('mean_price_per_c').reset_index()
# figure out the customers preferred index group name by looking at past transactions articles' index_group_name
ttransactions = pd.merge(transactions, articles[['article_id', 'index_group_name']], on='article_id', how='left')
# per customer count the number of purchases of each index_group_name
c2index_group_name = ttransactions.groupby('customer_id')['index_group_name'].value_counts().rename('count').reset_index()
c2index_group_name = c2index_group_name.sort_values('count', ascending=False).groupby('customer_id').head(1).rename(columns={'index_group_name': 'highest_count_ign_per_c'})
# merge the mean_price_per_c and c2index_group_name dataframes
t = pd.merge(c2index_group_name[['customer_id', 'highest_count_ign_per_c']], mean_price_per_c, on='customer_id', how='left')

# per article find the average age of the customers who bought it
transactions_with_age = pd.merge(transactions, customers[['customer_id', 'age']], on='customer_id', how='left')
mean_age_per_a = transactions_with_age.groupby('article_id')['age'].mean().rename('mean_age_per_a').astype('int8').reset_index()

# merge to get the mean_price_per_c, highest_count_ign and mean_age_per_a columns
transactions_with_3feat = pd.merge(transactions_with_age, t, on='customer_id', how='left')
transactions_with_3feat = pd.merge(transactions_with_3feat, mean_age_per_a, on='article_id', how='left')
transactions_with_3feat = pd.merge(transactions_with_3feat, articles[['article_id', 'index_group_name']], on='article_id', how='left')

In [105]:
transactions_with_3feat

,t_dat,customer_id,article_id,price,sales_channel_id,week,age,highest_count_ign_per_c,mean_price_per_c,mean_age_per_a,index_group_name
0,2020-07-15,272412481300040,778064028,0.008458,1,95,48,0,0.030597,32,0
1,2020-07-15,272412481300040,816592008,0.016932,1,95,48,0,0.030597,37,2
2,2020-07-15,272412481300040,621381021,0.033881,1,95,48,0,0.030597,35,2
3,2020-07-15,272412481300040,817477003,0.025407,1,95,48,0,0.030597,33,0
4,2020-07-15,272412481300040,899088002,0.025407,1,95,48,0,0.030597,35,2
...,...,...,...,...,...,...,...,...,...,...,...
2762867,2020-09-22,18439937050817258297,891591003,0.084729,2,104,49,0,0.040379,41,0
2762868,2020-09-22,18439937050817258297,869706005,0.084729,2,104,49,0,0.040379,39,0
2762869,2020-09-22,18440902715633436014,918894002,0.016932,1,104,18,0,0.028232,30,0
2762870,2020-09-22,18440902715633436014,761269001,0.016932,1,104,18,0,0.028232,33,0


In [106]:
t

,customer_id,highest_count_ign_per_c,mean_price_per_c
0,18442606110317830291,0,0.021770
1,371834049679164774,0,0.032494
2,12416564893706204993,0,0.036745
3,17234420386288341054,0,0.031118
4,979392728188584892,0,0.021122
...,...,...,...
437360,6420757378897806626,0,0.084729
437361,6420635653313041923,2,0.022017
437362,6420532178711353262,2,0.033881
437363,13570978177308658659,2,0.010153


In [107]:
# For every unique customer_id in the transactions_with_3feat dataframe, rank all the unique articles present in the transactions_with_3feat dataframe based on the difference between the customer's mean_price_per_c and the article's price. Then select the 100 best candidates of this ranking. Do this using built-in pandas functions. This will result in a new dataframe with 100*number_of_unique_customers rows.



In [108]:
# # Calculate the price difference for each article and customer
# candidates_3feat_price = (
#     transactions_with_3feat
#     .assign(price_difference=lambda x: abs(x['mean_price_per_c'] - x['price']))
# )
# 
# # Rank the articles within each customer group based on price difference
# candidates_3feat_price['price_rank'] = (
#     candidates_3feat_price
#     .groupby('customer_id')['price_difference']
#     .rank(ascending=True, method='min')
# )
# 
# # Select the top 100 candidates for each customer
# top_candidates_3feat_price = (
#     candidates_3feat_price
#     .loc[candidates_3feat_price.groupby('customer_id')['price_rank'].head(100).index]
# )
# 
# # Drop unnecessary columns
# top_candidates_3feat_price = top_candidates_3feat_price.drop(columns=['price_difference', 'price_rank'])

In [109]:
# Create a DataFrame with unique articles and their prices
unique_articles = transactions_with_3feat[['t_dat', 'article_id', 'price', 'index_group_name']].drop_duplicates()
# in unique_articles dataframe, if an article_id is not unique select the article_id with latest t_dat, drop the other rows with the same article_id
unique_articles = unique_articles.sort_values('t_dat', ascending=False).drop_duplicates('article_id').drop(columns=['t_dat'])

In [110]:
# # Merge with customer information
# candidates_3feat_price = pd.merge(
#     unique_articles,
#     mean_price_per_c[['customer_id', 'mean_price_per_c']],
#     how='cross'  # Cartesian product to get all combinations of articles and customers
# )
# 
# # Calculate the price difference for each combination
# candidates_3feat_price['price_difference'] = abs(candidates_3feat_price['mean_price_per_c'] - candidates_3feat_price['price'])
# 
# # Rank articles within each customer group based on price difference
# candidates_3feat_price['price_rank'] = (
#     candidates_3feat_price
#     .groupby('customer_id')['price_difference']
#     .rank(ascending=True, method='min')
# )
# 
# # Select the top 100 candidates for each customer
# top_candidates_3feat_price = (
#     candidates_3feat_price
#     .loc[candidates_3feat_price.groupby('customer_id')['price_rank'].head(100).index]
# )
# 
# # Drop unnecessary columns
# top_candidates_3feat_price = top_candidates_3feat_price.drop(columns=['price_difference', 'price_rank'])

In [ ]:
chunk_size = 1000  # Define the number of customers to process at once

unique_customers = mean_price_per_c['customer_id'].unique()[:1000]

# Process customers in chunks
chunks = [unique_customers[i:i + chunk_size] for i in range(0, len(unique_customers), chunk_size)]

result_candidates_3feat_price = pd.DataFrame()  # DataFrame to store final candidates

result_candidates_3feat_price_chunks = []

for customer_chunk in chunks:
    # Get articles for the current chunk of customers
    candidate_articles = pd.merge(
        unique_articles,
        pd.DataFrame({'customer_id': customer_chunk}),
        how='cross'
    )
        
    candidate_articles = pd.merge(candidate_articles, t, on='customer_id', how='left')
    candidate_articles = pd.merge(candidate_articles, mean_age_per_a, on='article_id', how='left')
    
    print(candidate_articles)

    # Calculate price difference for each combination
    candidate_articles['price_difference'] = abs(candidate_articles['mean_price_per_c'] - candidate_articles['price'])
    # Rank articles within each customer group based on price difference
    candidate_articles['price_rank'] = (
        candidate_articles
        .groupby('customer_id')['price_difference']
        .rank(ascending=True, method='min')
    )
    # Select the top 100 candidates for each customer
    top_candidates = (
        candidate_articles
        .sort_values(by=['customer_id', 'price_rank'])
        .groupby('customer_id')
        .head(100)
    )
    
    # Select 50 candidates where highest_count_ign_per_c is equal to index_group_name
    top_candidates = top_candidates[top_candidates['highest_count_ign_per_c'] == top_candidates['index_group_name']].head(50)
    
    # # Calculate age difference for each combination
    # top_candidates['age_difference'] = abs(top_candidates['age'] - top_candidates['mean_age_per_a'])
    # # Rank articles within each customer group based on age difference
    # top_candidates['age_rank'] = (
    #     top_candidates
    #     .groupby('customer_id')['age_difference']
    #     .rank(ascending=True, method='min')
    # )
    # # Select the top 25 candidates for each customer based on age difference
    # top_candidates = (
    #     top_candidates
    #     .sort_values(by=['customer_id', 'age_rank'])
    #     .groupby('customer_id')
    #     .head(25)
    # )
    
    # Append current chunk's candidates to result
    result_candidates_3feat_price_chunks.append(top_candidates)
    
# Concatenate all chunks into the final result    
result_candidates_3feat_price = pd.concat(result_candidates_3feat_price_chunks, ignore_index=True)

# Drop unnecessary columns
# top_candidates_3feat_price = result_candidates_3feat_price.drop(columns=['price_difference', 'price_rank'])
top_candidates_3feat_price = result_candidates_3feat_price.drop(columns=['price_difference'])

          article_id     price  index_group_name        customer_id  \
0          875719001  0.033881                 2     28847241659200   
1          875719001  0.033881                 2     41318098387474   
2          875719001  0.033881                 2    116809474287335   
3          875719001  0.033881                 2    200292573348128   
4          875719001  0.033881                 2    248294615847351   
...              ...       ...               ...                ...   
38330995   751336001  0.030492                 1  42872089953753496   
38330996   751336001  0.030492                 1  42878316430623896   
38330997   751336001  0.030492                 1  42910407533866495   
38330998   751336001  0.030492                 1  42942643248675808   
38330999   751336001  0.030492                 1  42984229297455520   

          highest_count_ign_per_c  mean_price_per_c  mean_age_per_a  
0                               0          0.046085              35  
1      

In [ ]:
top_candidates_3feat_price

In [ ]:
top_candidates_3feat_price[top_candidates_3feat_price['customer_id']==272412481300040]

In [ ]:
### Bestsellers candidates
mean_price = transactions.groupby(['week', 'article_id'])['price'].mean()
sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank').astype('int8')
bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week.week += 1

In [ ]:
# bestsellers_previous_week.pipe(lambda df: df[df['week']==96])
bestsellers_previous_week

In [ ]:
unique_transactions = transactions.groupby(['week', 'customer_id']).head(1) .drop(columns=['article_id', 'price']).copy()
candidates_bestsellers = pd.merge(unique_transactions, bestsellers_previous_week, on='week',)
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week

In [ ]:
# candidates_bestsellers

In [ ]:
# test_set_transactions
# unique_transactions

In [ ]:
candidates_bestsellers_test_week = pd.merge(test_set_transactions, bestsellers_previous_week, on='week')
candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)

In [ ]:
candidates_bestsellers

# Combining transactions and candidates / negative examples

In [ ]:
transactions['purchased'] = 1
data = pd.concat([transactions, candidates_last_purchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)
data.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)

In [ ]:
data

### Add bestseller information

In [ ]:
data = pd.merge(
    data,
    bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'],
    how='left'
)
data = data[data.week != data.week.min()]
data.bestseller_rank.fillna(999, inplace=True)
data = pd.merge(data, articles, on='article_id', how='left')
data = pd.merge(data, customers, on='customer_id', how='left')
data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [ ]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()

In [ ]:
test

In [ ]:
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

In [ ]:
extra_columns = []
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'bestseller_rank']
columns_to_use.extend(extra_columns)

In [ ]:
train_X = train[columns_to_use]
train_y = train['purchased']

test_X = test[columns_to_use]

In [ ]:
test_X

# Model training

In [ ]:
from lightgbm.sklearn import LGBMRanker
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=1,
    importance_type='gain',
    verbose=10
)
ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
)

In [ ]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

# Calculate predictions

In [ ]:
test['preds'] = ranker.predict(test_X)

c_id2predicted_article_ids = test \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()

bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()

# Create submission

In [ ]:
sub = pd.read_csv('../input/sample_submission.csv')
preds = []
for c_id in customer_hex_id_to_int(sub.customer_id):
    pred = c_id2predicted_article_ids.get(c_id, [])
    pred = pred + bestsellers_last_week
    preds.append(pred[:12])
preds = [' '.join(['0' + str(p) for p in ps]) for ps in preds]
sub.prediction = preds
sub_name = 'basic_model_submission'
sub.to_csv(f'{sub_name}.csv.gz', index=False)